In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE107846"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE107846"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE107846.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE107846.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE107846.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Secondhand smoke alters arachidonic acid metabolism in infants and children with cystic fibrosis"
!Series_summary	"Children ages 0-10 years old with CF were recruited from 2012-2015 at the outpatient CF clinic, and classified according to age (infants <1 year old, vs. children 1-10 years old). The diagnosis of CF was defined as two disease-causing mutations or a sweat chloride test ≥ 60 mmol/L. Hair and blood samples were collected from each subject. Hair nicotine concentrations were determined and considered as the primary objective measure of SHSe.  Hair nicotine provides a long-term measure of SHSe as nicotine is integrated into the growing hair shaft over multiple months. (15)  For each subject, 30-40 shafts of hair of approximately 2-3 cm in length were cut at the hair root from the occipital skull.  Hair samples were refrigerated at 4° for storage, washed before analyses to remove ambient nicotine (15) and batch-tested at a contract research

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the series title and summary, this appears to be a gene expression study
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (CF status)
trait_row = 5  # 'state: Healthy' or 'state: CF'

# For age
age_row = 1  # 'age: X' where X is the age in years

# For gender
gender_row = 2  # 'Sex: F' or 'Sex: M'

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert CF status to binary (0 for healthy, 1 for CF)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if 'state:' in value:
        actual_value = value.split('state:')[1].strip()
        if 'cf' in actual_value.lower():
            return 1  # CF
        elif 'healthy' in actual_value.lower():
            return 0  # Healthy
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    if 'age:' in value:
        try:
            age_str = value.split('age:')[1].strip()
            age_val = float(age_str)
            return age_val
        except (ValueError, IndexError):
            return None
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if 'sex:' in value:
        gender = value.split('sex:')[1].strip().lower()
        if gender == 'f':
            return 0  # Female
        elif gender == 'm':
            return 1  # Male
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Define clinical data based on the sample characteristics dictionary
    clinical_data = pd.DataFrame({
        0: ['identifier: NCH-C019', 'identifier: NCH-C011', 'identifier: NCH-C010', 'identifier: NCH-C017', 'identifier: NCH-C016', 'identifier: NCH-C008', 'identifier: NCH-C001', 'identifier: NCH-C009', 'identifier: NCH-C015', 'identifier: NCH-C020', 'identifier: NCH-C012', 'identifier: NCH-C013', 'identifier: CF-050', 'identifier: CF-044', 'identifier: CF-027', 'identifier: CF-028', 'identifier: CF-033', 'identifier: CF-026', 'identifier: CF-052', 'identifier: CF-058', 'identifier: CF-021', 'identifier: CF-018', 'identifier: CF-060', 'identifier: CF-031', 'identifier: CF-063', 'identifier: CF-049', 'identifier: CF-034', 'identifier: CF-048', 'identifier: CF-065', 'identifier: CF-030'],
        1: ['age: 9', 'age: 3.8', 'age: 5.1', 'age: 3.4', 'age: 7', 'age: 2.8', 'age: 4.3', 'age: 2.3', 'age: 9.9', 'age: 7.8', 'age: 7.25', 'age: 4', 'age: 2.333', 'age: 1.917', 'age: 8.583', 'age: 6.8', 'age: 2.667', 'age: 9.917', 'age: 1.083', 'age: 2.25', 'age: 7.75', 'age: 6.833', 'age: 4.583', 'age: 6.417', 'age: 4.75', 'age: 4.333', 'age: 5.25', 'age: 4.25', 'age: 6', 'age: 6.167'],
        2: ['Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M', 'Sex: F', 'Sex: M'],
        5: ['state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: Healthy', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF', 'state: CF']
    })
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{0: [nan, nan, nan], 1: [nan, 3.8, nan], 2: [nan, nan, 0.0], 5: [0.0, nan, nan]}
Clinical data saved to ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE107846.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 72
Header line: "ID_REF"	"GSM2881538"	"GSM2881539"	"GSM2881540"	"GSM2881541"	"GSM2881542"	"GSM2881543"	"GSM2881544"	"GSM2881545"	"GSM2881546"	"GSM2881547"	"GSM2881548"	"GSM2881549"	"GSM2881550"	"GSM2881551"	"GSM2881552"	"GSM2881553"	"GSM2881554"	"GSM2881555"	"GSM2881556"	"GSM2881557"	"GSM2881558"	"GSM2881559"	"GSM2881560"	"GSM2881561"	"GSM2881562"	"GSM2881563"	"GSM2881564"	"GSM2881565"	"GSM2881566"	"GSM2881567"	"GSM2881568"	"GSM2881569"	"GSM2881570"	"GSM2881571"	"GSM2881572"	"GSM2881573"	"GSM2881574"	"GSM2881575"	"GSM2881576"	"GSM2881577"
First data line: "ILMN_1343291"	17217.2773	17467.6289	20416.6211	19897.6895	18640.1309	18753.8438	20050.9199	17575.5664	16891.3438	19166.2129	18103.623	17737.4863	21254.9785	21347.5156	19289.6367	18520.1738	17510.5781	19875.5879	18027.125	19387.7383	17609.2246	19102.0215	19656.0996	21552.1309	21019.0957	21135.6445	18077.4922	17769.8301	20254.6035	19082.5742	21348.9824	19946.4727	18564.8184	17811.8359	20162.0098	19104.7461	206

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they are Illumina BeadArray 
# probe identifiers, not human gene symbols
# These need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1762337', 'ILMN_2055271', 'ILMN_1736007', 'ILMN_2383229', 'ILMN_1806310'], 'SPECIES': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'SOURCE': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'SEARCH_KEY': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_138933.1'], 'TRANSCRIPT': ['ILMN_183371', 'ILMN_175569', 'ILMN_18893', 'ILMN_18532', 'ILMN_7300'], 'ILMN_GENE': ['7A5', 'A1BG', 'A1BG', 'A1CF', 'A1CF'], 'SOURCE_REFERENCE_ID': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'REFSEQ_ID': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENTREZ_GENE_ID': [346389.0, 1.0, 1.0, 29974.0, 29974.0], 'GI': [47271497.0, 21071029.0, 21071029.0, 20357574.0, 20357571.0], 'ACCESSION': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'SYMBOL': ['7A5', 'A1BG', 'A1BG', 'A1CF', 'A1CF'], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the appropriate columns for mapping
# Based on the gene annotation preview, we can see that:
# - 'ID' column in the annotation dataframe contains the gene identifiers (ILMN_*)
# - 'SYMBOL' column contains the gene symbols (e.g., A1BG)

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'SYMBOL')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression dataframe shape after normalization: {gene_data.shape}")
print(f"First 5 normalized gene symbols: {gene_data.index[:5].tolist()}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44053, 2)
Gene expression dataframe shape after mapping: (21464, 40)
First 5 gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1']


Gene expression dataframe shape after normalization: (20259, 40)
First 5 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']


Gene expression data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE107846.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features directly from the matrix file
try:
    # Get the file paths for the matrix file to extract clinical data
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get raw clinical data from the matrix file
    _, clinical_raw = get_background_and_clinical_data(matrix_file)
    
    # Verify clinical data structure
    print("Raw clinical data shape:", clinical_raw.shape)
    
    # Extract clinical features using the defined conversion functions
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical features:")
    print(clinical_features)
    
    # Save clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    print(f"  Genes with >20% missing: {sum(linked_data.iloc[:, 1:].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data.iloc[:, 1:].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data comparing CFTR WT vs CFTR mutant (p.Phe508del) samples."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20259, 40)
First few genes with their expression values after normalization:
          GSM2881538  GSM2881539  GSM2881540  GSM2881541  GSM2881542  \
Gene                                                                   
A1BG          8.7476     21.9606     23.2725     15.6942     29.2452   
A1BG-AS1     14.6337      4.9228     -4.9448      7.6773      3.3042   
A1CF        -14.7669      3.2132     15.3795     -1.3697     -1.3926   
A2M          -8.5760     -2.6491     -3.7892     -8.4301     -7.5382   
A2ML1        -7.3726    -11.7410    -10.4285     -2.1618     -9.3990   

          GSM2881543  GSM2881544  GSM2881545  GSM2881546  GSM2881547  ...  \
Gene                                                                  ...   
A1BG         25.1912     36.4149      3.4988      7.2743      7.5244  ...   
A1BG-AS1     12.0664      2.6297     -0.5639      9.5417     -0.9727  ...   
A1CF          3.5824      2.9546      2.3193     -8.2343     10.8697  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE107846.csv
Raw clinical data shape: (7, 41)
Clinical features:
                 GSM2881538  GSM2881539  GSM2881540  GSM2881541  GSM2881542  \
Cystic_Fibrosis         0.0         0.0         0.0         0.0         0.0   
Age                     9.0         3.8         5.1         3.4         7.0   
Gender                  0.0         0.0         1.0         0.0         1.0   

                 GSM2881543  GSM2881544  GSM2881545  GSM2881546  GSM2881547  \
Cystic_Fibrosis         0.0         0.0         0.0         0.0         0.0   
Age                     2.8         4.3         2.3         9.9         7.8   
Gender                  1.0         0.0         0.0         0.0         1.0   

                 ...  GSM2881568  GSM2881569  GSM2881570  GSM2881571  \
Cystic_Fibrosis  ...        1.00         1.0       1.000       1.000   
Age              ...        4.25         6.0       6.167       2.417   
Gend

Data shape after handling missing values: (40, 20262)
For the feature 'Cystic_Fibrosis', the least common label is '0.0' with 12 occurrences. This represents 30.00% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 3.25
  50% (Median): 4.7085
  75%: 6.80825
Min: 1.083
Max: 9.917
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Cystic_Fibrosis/cohort_info.json


Linked data saved to ../../output/preprocess/Cystic_Fibrosis/GSE107846.csv
